# Reserve and configure FABRIC resources for “setting a network of Autonomous systems”

In the experiment setup, we try to emulate the real world network by designing a network of various autonomous systems spread across different sites available on FABRIC testbed.

## Set up your FABRIC environment

This assumes that you have already configured your FABRIC account and your Jupyter environment as described in [Hello, FABRIC](https://teaching-on-testbeds.github.io/blog/hello-fabric).

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

In [ ]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Create and submit a slice

In [ ]:
slice_name="network-"+ fablib.get_bastion_username()
slice = fablib.new_slice(name=slice_name)

### Set up the variables

Next, we will set up the variables- number of autonomous sytems in the network, number of routers in each autonomous system, whether a system in CDN or not and the hops at which redundancy can be seen.

In [ ]:
import random
#number of routers in access network similar irrespective of cdn or not
#redundancy_AS_1 parameter sets the hop at which redundant paths will be set
hops_AS_1=random.randint(3,6)
redundancy_AS_1=3

# For cdn, define
# number of autonomous systems including source and target AS (ranges between 2-4)
# hops seen in destination network (ranges between 3-4)
# hops seen in intermediate network (ranges bewteen 1-2)

number_of_AS_cdn=random.randint(2,4)
hops_final_AS_cdn=random.randint(3,4)
max_hops_intermediate_cdn=3

# For non CDN, define
# number of autonomous systems including source and target AS (ranges between 3-5)
# hops seen in destination network (ranges between 5-8)
# hops seen in intermediate network (ranges bewteen 1-4)

number_of_AS_not_cdn=random.randint(3,5)
hops_final_AS_not_cdn=random.randint(5,8)
max_hops_intermediate_not_cdn=4

# Set cdn=True if CDN type opf network is required else False
cdn=True
sites=['NCSA', 'GATECH', 'WASH', 'GPN', 'INDI', 'SALT', 'CERN', 'MICH', 'DALL', 'STAR', 'EDC', 'PSC']

Print all the parameters that were setup randomly.

In [ ]:
# Print all the parameters:
print("Hops in first AS: ", hops_AS_1)
print("Number of AS: ", number_of_AS_cdn if cdn else number_of_AS_not_cdn)
print("Hops in final AS: ", hops_final_AS_cdn if cdn else hops_final_AS_not_cdn )

### Set up the routers and networks

First we will create a function that will generate an autonomous system along with the node configuration, net configuration, resource requirement data.

In [ ]:

def create_AS(as_num, n_hops, r):
    routers=[1]*min(r, n_hops) + [random.randint(2,3) for i in range(max(0, n_hops - r - 1))] + [1 for i in range(0,1) if n_hops!=r]
    node_conf = [ {'name': "as" + str(as_num) + '-r-' + str(j) + '-' + str(i) ,
        'cores': 8, 'ram': 16, 'disk': 25, 'image': 'default_ubuntu_22', 'packages': ['net-tools']}
         for j,level in enumerate(routers) for i in range(level)]
    r_mul=[0]+[routers[i-1]*routers[i] for i in range(1,len(routers))]
    net_conf = [
     {"name": "as"+str(as_num)+"-net"+str(j)+str(i)+"-"+str(j+1)+str(k), "subnet": "10."+str(as_num)+"."+str(sum(r_mul[:j+1])+(routers[j+1]*i)+k+1)+".0/24", 
      "nodes": [{"name": "as"+str(as_num)+'-r-' + str(j) + '-' + str(i),   "addr": "10."+str(as_num)+"."+str(sum(r_mul[:j+1])+(routers[j+1]*i)+k+1)+"."+str(1)} ]+ 
      [{"name": "as"+str(as_num)+'-r-' + str(j+1) + '-' + str(k),   "addr": "10."+str(as_num)+"."+str(sum(r_mul[:j+1])+(routers[j+1]*i)+k+1)+"."+str(2)}]}
            for j,level in enumerate(routers[:-1]) for i in range(level) for k in range(routers[j+1])]
    exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes'])+2 for n in net_conf]) }
    return [routers, node_conf, net_conf, exp_conf]


Next, we will call create_AS() function to generate the number of autonomous systems which was defined in one of the previous cell.

In [ ]:
if cdn:
    number_of_AS=number_of_AS_cdn
    hops_final_AS=hops_final_AS_cdn
    max_hops_intermediate=max_hops_intermediate_cdn
    
else:    
    number_of_AS=number_of_AS_not_cdn
    hops_final_AS=hops_final_AS_not_cdn
    max_hops_intermediate=max_hops_intermediate_not_cdn


# set up the number of routers in first AS

data_routers=[create_AS(0, hops_AS_1, redundancy_AS_1)]

  
# set up the number of routers in intermediate AS
# hops in intermediate AS is randomly selected between 1 and max_hops_intermediate value
# redundant path can start at any value between 2nd hop and hops_intermediate-1 in the intermediate AS

for i in range(1, number_of_AS-1):
    hops_intermediate=random.randint(1,max_hops_intermediate)
    print(hops_intermediate)
    data_routers+=[create_AS(i, hops_intermediate, r=random.randint(1,hops_intermediate))]

# set up the number of routers in Destination AS
# redundant path can start at any value between 2nd hop and hops_final_AS-1 in the destination AS
data_routers+=[create_AS(number_of_AS-1, hops_final_AS, random.randint(1,hops_final_AS))]


In [ ]:
node_conf = [node for as_conf in data_routers for node in as_conf[1]]
node_conf

In [ ]:
net_conf = [net for as_conf in data_routers for net in as_conf[2]]
net_conf

In [ ]:
as_net_conf=[{"router-1": r[1][-1]['name'], "router-2":data_routers[i+1][1][0]['name'], "net-name": "ext-net"+str(i)+"-"+str(i+1)} for i, r in enumerate(data_routers[:-1])]

In [ ]:
as_net_conf

### Reserve resources

Now we will get a list of sites that has sufficient resources for the experiment.

In [ ]:

# Get the sites
as_sites = []
for i,r in enumerate(data_routers):   
    while True:
        site_name = fablib.get_random_site(avoid=sites + as_sites)
        if ( (fablib.resources.get_core_available(site_name) > 1.2*r[3]['cores']) and
            (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**r[3]['nic']) ):
            break

    print(f"AS {i} will use {site_name}")
    #fablib.show_site(site_name)
    as_sites.append(site_name)


In [ ]:
print(as_sites)

Then we will add hosts and network segments

In [ ]:
for i,r in enumerate(data_routers):
     for rtr in r[1]: # iterate over the nodes_conf for this AS
        slice.add_node(name=rtr['name'], site=as_sites[i], 
                       cores=rtr['cores'], 
                       ram=rtr['ram'], 
                       disk=rtr['disk'], 
                       image=rtr['image'])

In [ ]:
print(slice.list_nodes())

In [ ]:
# this cell sets up the network links 
for i,r in enumerate(data_routers):
     for net in r[2]: # iterate over net_conf for each AS
        ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                     name=net["name"]).get_interfaces()[0] for node in net['nodes'] ]
        slice.add_l2network(name=net["name"], type='L2Bridge', interfaces=ifaces)

for i,r in enumerate(as_net_conf):
    iface1 = slice.get_node(r['router-1']).add_component(model='NIC_Basic', name=r['net-name']).get_interfaces()[0]
    iface2 = slice.get_node(r['router-2']).add_component(model='NIC_Basic', name=r['net-name']).get_interfaces()[0]
    slice.add_l2network(name=r['net-name'], interfaces=[iface1, iface2])

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes. While it is being prepared, the “State” of the slice will appear as “Configuring”. When it is ready, the “State” of the slice will change to “StableOK”.

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

### Set up the routes

We will then install FRR routing software on all of the routers, and then configure to use OSPF as their internal routing protocol.

In [ ]:
router_nodes=[slice.get_node(name=r['name']) for r in node_conf]


In [ ]:

for n in router_nodes:
    n.execute("curl -s https://deb.frrouting.org/frr/keys.asc | sudo apt-key add -")
    n.execute("echo deb https://deb.frrouting.org/frr $(lsb_release -s -c) frr-stable | sudo tee -a /etc/apt/sources.list.d/frr.list")
    n.execute("sudo apt update")
    n.execute("sudo apt -y install frr frr-pythontools nload")
    n.execute("sudo sed -i 's/ospfd=no/ospfd=yes/g' /etc/frr/daemons")
    n.execute("sudo sed -i 's/bgpd=no/bgpd=yes/g' /etc/frr/daemons")
    n.execute("sudo systemctl restart frr.service")    

In [ ]:

for n in router_nodes:
    n.execute("sudo vtysh -E -c'configure terminal\nrouter ospf\nnetwork 10."+n.get_name()[2]+".0.0/16 area 0.0.0.0\n exit\n exit\n exit'  ")    

In [ ]:

for n in router_nodes:
    n.execute("sudo vtysh -E -c 'show ip route\nexit'")   

Validate the routing by running ping across the network

In [ ]:
for r in data_routers:
    print([i['addr'] for i in r[2][-1]['nodes']])
    [(slice.get_node(name=r[1][0]['name']).execute("ping -c 5 "+i['addr'] +" | grep rtt")) for i in r[2][-1]['nodes']]  

#### Draw the network topology

The following cell will draw the network topology, for your reference.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node']) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

### Delete your slice

When you finish your experiment, you should delete your slice! The following cells deletes all the resources in your slice, freeing them for other experimenters.

In [ ]:
slice = fablib.get_slice(name=slice_name)
fablib.delete_slice(slice_name)

In [ ]:
# slice should end up in "Dead" state
# re-run this cell until you see it in "Dead" state
slice.update()
_ = slice.show()